# Introduction to Context Providers

In the previous notebook, you built an agent that uses **tools** — functions the agent explicitly decides to call. In this notebook, you'll learn about **context providers**, MAF's mechanism for automatically injecting context before each agent invocation.

## Tools vs Context Providers

| | Tools | Context Providers |
|---|---|---|
| **When they run** | Only when the agent decides to call them | Automatically before **every** agent invocation |
| **How they work** | Agent sees function name + docstring, decides to call | `before_run()` injects instructions/messages into `SessionContext` |
| **Best for** | On-demand actions (lookups, API calls) | Always-available background knowledge (RAG, memory, user preferences) |

## Context Provider Lifecycle

Context providers have two lifecycle hooks that run around each agent invocation:

```
agent.run(query)
    │
    ├── before_run()  ← Inject context into SessionContext
    │       • context.extend_instructions()  — add to system prompt
    │       • context.extend_messages()       — add messages to context
    │       • context.extend_tools()          — dynamically add tools
    │
    ├── LLM processes query + injected context
    │
    └── after_run()   ← Process the response
            • Extract structured data from the conversation
            • Store information in session.state for next turn
```

## What You'll Build

A `UserInfoMemory` context provider that:
- **`before_run()`** — Injects dynamic instructions based on what it knows about the user
- **`after_run()`** — Extracts structured data (name, age) from the conversation using the LLM

No external services are needed beyond the LLM. In Lab 6, you'll use the same pattern with `Neo4jContextProvider` to inject knowledge graph results.

***

Load the environment variables and import the required Python modules.

In [ ]:
import sys
sys.path.insert(0, '../shared')

from typing import Any

from agent_framework import (
    AgentSession,
    BaseContextProvider,
    SessionContext,
    SupportsChatGetResponse,
)
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential
from pydantic import BaseModel

from config import get_agent_config

Get the agent configuration from the environment.

In [ ]:
config = get_agent_config()

## Define the Data Model

Define a Pydantic model for the structured data the context provider will extract. The LLM will use this schema to return structured output.

In [ ]:
class UserInfo(BaseModel):
    name: str | None = None
    age: int | None = None

## Define the Context Provider

Create a `UserInfoMemory` context provider that inherits from `BaseContextProvider`.

The two key methods are:

- **`before_run()`** — Called before each agent invocation. Receives the `SessionContext` and uses `context.extend_instructions()` to inject dynamic instructions based on what it knows about the user.
- **`after_run()`** — Called after each agent invocation. Uses `context.get_messages()` to read the conversation, then makes a structured LLM call to extract the user's name and age.

State is stored in `session.state["user-info-memory"]` — a persistent dictionary that survives across conversation turns. Each context provider gets its own namespace in this state dict via `self.source_id`.

In [ ]:
class UserInfoMemory(BaseContextProvider):
    """Context provider that extracts and remembers user info (name, age)."""

    def __init__(self, client: SupportsChatGetResponse):
        super().__init__("user-info-memory")
        self._chat_client = client

    async def before_run(
        self,
        *,
        agent: Any,
        session: AgentSession | None,
        context: SessionContext,
        state: dict[str, Any],
    ) -> None:
        """Inject dynamic instructions based on stored user info."""
        my_state = state.setdefault(self.source_id, {})
        user_info = my_state.setdefault("user_info", UserInfo())

        instructions: list[str] = []

        if user_info.name is None:
            instructions.append(
                "Ask the user for their name and politely decline to answer any questions until they provide it."
            )
        else:
            instructions.append(f"The user's name is {user_info.name}.")

        if user_info.age is None:
            instructions.append(
                "Ask the user for their age and politely decline to answer any questions until they provide it."
            )
        else:
            instructions.append(f"The user's age is {user_info.age}.")

        context.extend_instructions(self.source_id, " ".join(instructions))

    async def after_run(
        self,
        *,
        agent: Any,
        session: AgentSession | None,
        context: SessionContext,
        state: dict[str, Any],
    ) -> None:
        """Extract user info from the conversation after each turn."""
        my_state = state.setdefault(self.source_id, {})
        user_info = my_state.setdefault("user_info", UserInfo())
        if user_info.name is not None and user_info.age is not None:
            return  # Already have everything

        request_messages = context.get_messages(include_input=True, include_response=True)
        user_messages = [
            msg for msg in request_messages
            if hasattr(msg, "role") and msg.role == "user"
        ]
        if not user_messages:
            return

        try:
            result = await self._chat_client.get_response(
                messages=request_messages,
                instructions="Extract the user's name and age from the message if present. "
                "If not present return nulls.",
                options={"response_format": UserInfo},
            )
            extracted = result.value
            if extracted and user_info.name is None and extracted.name:
                user_info.name = extracted.name
            if extracted and user_info.age is None and extracted.age:
                user_info.age = extracted.age
            state.setdefault(self.source_id, {})["user_info"] = user_info
        except Exception:
            pass  # Failed to extract, continue without updating

> Notice how the context provider uses `context.extend_instructions()` to dynamically modify the agent's behavior each turn — this is the same mechanism that Neo4j context providers use in Lab 6 to inject knowledge graph results.

***

## Create the Agent

Create an agent with the `UserInfoMemory` context provider. The agent doesn't need any tools — the provider handles information extraction automatically.

In [ ]:
credential = AzureCliCredential()
client = AzureOpenAIResponsesClient(
    project_endpoint=config.project_endpoint,
    deployment_name=config.model_name,
    credential=credential,
)

agent = client.as_agent(
    name="workshop-context-provider-agent",
    instructions="You are a friendly assistant. Always address the user by their name when you know it.",
    context_providers=[UserInfoMemory(client)],
)

session = agent.create_session()

async def ask(query):
    """Run a query against the agent and print the response."""
    print(f"User: {query}\n")
    print("Assistant: ", end="", flush=True)
    response = await agent.run(query, session=session)
    print(response.text)
    print()

print(f"Agent '{agent.name}' created")

## Multi-Turn Conversation

Run the cells below to see the context provider in action. Watch how the agent's behavior changes as it learns about the user.

**Turn 1** — The provider has no user info yet, so it injects instructions to ask for the user's name and age.

In [ ]:
await ask("Hello, what is the square root of 9?")

**Turn 2** — The user provides their name. After this turn, `after_run()` extracts it using structured LLM output.

In [ ]:
await ask("My name is Alex")

**Turn 3** — The user provides their age. The provider now has all the information it needs.

In [ ]:
await ask("I am 30 years old")

**Turn 4** — Now that the provider knows the user, it injects their name and age as instructions. The agent can answer freely and uses the user's name.

In [ ]:
await ask("Now, what is the square root of 9?")

***

## Inspect Session State

The context provider stores extracted data in `session.state`. This state persists across turns and can be serialized for long-term storage.

In [ ]:
user_info = session.state.get("user-info-memory", {}).get("user_info", UserInfo())
print(f"Extracted Name: {user_info.name}")
print(f"Extracted Age: {user_info.age}")

> The context provider automatically extracted structured data from the conversation — without the agent needing to call any tools. In Lab 6, you'll use this same pattern with `Neo4jContextProvider` to automatically inject knowledge graph context before each agent invocation.

***

[Continue to Lab 6 - MAF Context Providers](../Lab_6_Context_Providers)

In [ ]:
# No cleanup needed — AzureOpenAIResponsesClient and sync credentials
# don't require explicit lifecycle management